## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
    
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [4]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [5]:
!ls ../../Data/Weather/

PRCP_residuals_PCA.pickle   decon_BBBSBBBB_SNWD.parquet
STAT_BBBSBBBB.pickle        decon_BBBSBBBB_TOBS.parquet
STAT_SSSBBSSB.pickle        decon_SSSBBSSB_PRCP.parquet
US_Weather_BBBSBBBB.csv     decon_SSSBBSSB_SNWD.parquet
US_Weather_BBBSBBBB.csv.gz  ghcnd-readme.txt
US_Weather_BBBSBBBB.parquet ghcnd-stations.txt
US_Weather_SSSBBSSB.csv     ghcnd-stations_buffered.txt
US_Weather_SSSBBSSB.csv.gz  ghcnd-version.txt
US_Weather_SSSBBSSB.parquet stations.pkl
data-source.txt             stations.pkl.gz
decon_BBBSBBBB.parquet


In [6]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBBSSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
#filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
499
+-------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|            coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|              res_2|              res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+-------------------+------------------+------------------+---------+--------+--------+---------+-----------+------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|-3975.9681279375172|2226.5193552251358|324.35763807368573|   2145.8|SSSBBSSB| 37.4403|-112.4819|       SNWD|0.3415303260823011| 0.1314942531736086|0.12569642685188925|0.761567673887

In [7]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
Query="SELECT station, latitude,longitude,elevation,%s FROM weather"%feature
print(Query)
df1 = sqlContext.sql(Query)
df1.show(4)

SELECT station, latitude,longitude,elevation,coeff_1 FROM weather
+-----------+--------+---------+---------+-------------------+
|    station|latitude|longitude|elevation|            coeff_1|
+-----------+--------+---------+---------+-------------------+
|USC00420086| 37.4403|-112.4819|   2145.8|-3975.9681279375172|
|USC00420900| 37.6933|-112.8469|   2977.9|-12587.643158893527|
|USC00420900| 37.6933|-112.8469|   2977.9|-12956.464704805785|
|USC00420900| 37.6933|-112.8469|   2977.9|-13492.556735554954|
+-----------+--------+---------+---------+-------------------+
only showing top 4 rows



In [8]:
df1

DataFrame[station: string, latitude: double, longitude: double, elevation: double, coeff_1: double]

In [9]:
df2=df1.groupby(['station','latitude','longitude','elevation']).agg({"station": "count", feature: "mean"})
pdf=df2.toPandas()
pdf.sort_values(by=['station'],inplace=True)
pdf.head(2)

station  latitude  longitude  elevation  count(station)  avg(coeff_1)
2   USC00260099   37.3667  -115.1667     -999.9               1    358.254596
37  USC00261358   37.6128  -114.5264     1334.7               4    219.785875

In [10]:
#define a mapping from the range of the value to hex colors.
from matplotlib.colors import rgb2hex
_avg='avg(%s)'%feature
_min=pdf[_avg].min()
_max=pdf[_avg].max()
_min,_max

import pylab as plt
cmap=plt.get_cmap('jet')
def get_color(val):
    x=(val-_min)/(_max-_min)
    return(rgb2hex(cmap(x)[:3]))

get_color(1000.)

/Users/toby/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


u'#800000'

## Map

In [11]:
#min_lat,max_lat,min_long,max_long = box = (42.1103, 42.6167, -72.6, -70.8)
min_lat,max_lat,min_long,max_long = box = \
(pdf['latitude'].min(), pdf['latitude'].max(), pdf['longitude'].min(), pdf['longitude'].max())

In [12]:
min_lat, max_lat, min_long, max_long

(37.272199999999998, 38.0, -115.16670000000001, -109.08280000000001)

In [41]:
marker = Marker(location=[37.5, -110])


In [17]:
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 6

url = "http://{s}.tile.opentopomap.org/{z}/{x}/{y}.png"
#url = 'http://{s}.tile.osm.org/{z}/{x}/{y}.png'
provider = TileLayer(url=url, opacity=1)
#provider = TileLayer(opacity=1)

m = Map(default_tiles=provider, center=center, zoom=zoom)

r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
circles = []
for index,row in pdf.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    _count=row['count(station)']
    _coef=row[_avg]
    # taking sqrt of count so that the  area of the circle corresponds to the count
    c = Circle(location=(_lat,_long), radius=int(300*np.sqrt(_count+0.0)), weight=1,
            color='#F00', opacity=0.8, fill_opacity=0.4,
            fill_color=get_color(_coef))
    marker = Marker(location=[_lat, _long], opacity=1)
    #circles.append(marker)
    m.add_layer(marker)
m.layout.height = '600px'
#m.add_layer(marker)
m    

In [18]:
url = "http://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}.png"
provider = TileLayer(url=url, opacity=1)

m = Map(default_tiles=provider, center=center, zoom=zoom)

r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
circles = []
for index,row in pdf.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    _count=row['count(station)']
    _coef=row[_avg]
    # taking sqrt of count so that the  area of the circle corresponds to the count
    c = Circle(location=(_lat,_long), radius=int(300*np.sqrt(_count+0.0)), weight=1,
            color='#F00', opacity=0.8, fill_opacity=0.4,
            fill_color=get_color(_coef))
    circles.append(c)
    m.add_layer(c)
m.layout.height = '800px'
m       

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [19]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');